Residential MCE Bill (Generation Charges Only)

"EV, Residential Rates for Electric Vehicle Owners" plus "Deep Green (Residential)"

In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [2]:
from cost.utility_rate.models import RatePlan
from load.customer.models import Meter

In [3]:
from datetime import datetime

# Step 1. Choose start and end_limit dates (Jan. 2 2018 through Jan. 30 2018)
start = datetime(2018, 1, 2)
end_limit = datetime(2018, 1, 31)

# Step 2. Get meter data
meter = Meter.objects.get(sa_id=8943913372)
intervalframe = meter.intervalframe.filter_by_datetime(start=start, end_limit=end_limit)

# Step 3. Get rates (OpenEI format)
ev_plan = RatePlan.objects.get(name="EV, Residential Rates for Electric Vehicle Owners")
ev_rates = ev_plan.get_latest_rate_collection(start)
ev_openei_rate_data = ev_rates.openei_rate_data

deep_green_plan = RatePlan.objects.get(name="Deep Green (Residential)")
deep_green_rates = deep_green_plan.get_latest_rate_collection(start)
deep_green_openei_rate_data = deep_green_rates.openei_rate_data

In [4]:
from beo_datastore.libs.bill import ValidationBill

# Step 4. Generate bills
ev_bill = ValidationBill(intervalframe=intervalframe, openei_rate_data=ev_openei_rate_data)
deep_green_bill = ValidationBill(intervalframe=intervalframe, openei_rate_data=deep_green_openei_rate_data)

display(ev_bill.total_dataframe)
display(deep_green_bill.total_dataframe)

,category,description,tou_period,count,count_unit,rate,rate_unit,pro_rata,total
0,energy,Energy Charge - Winter Peak,3,88.57,kWh,0.055,$/kWh,1,4.87135
1,energy,Energy Charge - Winter Part-Peak,4,132.59,kWh,0.03,$/kWh,1,3.97770
2,energy,Energy Charge - Winter Off-Peak,5,585.02,kWh,0.03,$/kWh,1,17.55060
Total,,,,806.18,,,,,26.39965


,category,description,tou_period,count,count_unit,rate,rate_unit,pro_rata,total
0,energy,Energy Charge - Deep Green $0.01/kWh,0,806.18,kWh,0.01,$/kWh,1,8.0618
Total,,,,806.18,,,,,8.0618


In [5]:
from beo_datastore.libs.bill import BillingCollection

# Step 5. Consolodate bills
bill = BillingCollection(bills=[ev_bill, deep_green_bill])

bill.total_dataframe

,category,description,tou_period,count,count_unit,rate,rate_unit,pro_rata,total
0,energy,Energy Charge - Winter Peak,3,88.57,kWh,0.055,$/kWh,1,4.87135
1,energy,Energy Charge - Winter Part-Peak,4,132.59,kWh,0.03,$/kWh,1,3.97770
2,energy,Energy Charge - Winter Off-Peak,5,585.02,kWh,0.03,$/kWh,1,17.55060
0,energy,Energy Charge - Deep Green $0.01/kWh,0,806.18,kWh,0.01,$/kWh,1,8.06180
Total,,,,1612.36,,,,,34.46145
